# The Backwards One Body Waveform: "BOB"

## Author: Siddharth Mahesh

## This module documents the Backwards One Body waveform calibrated to numerical fits from pySEOBNR's SEOBNRv5HM gravitational waveform approximant.


**Notebook Status:** <font color='red'><b> In Progress? </b></font>

**Validation Notes:** 

### NRPy+ Source Code for this module: [v5HM-BOB_unoptimized_merger_ringdown.py](./Radiation/v5HM-BOB_unoptimized_merger_ringdown.py)

<a id='intro'></a>

## Introduction
$$\label{intro}$$

### The Physical System of Interest

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${S}_{1}$, ${S}_{2}$ that are aligned (i.e parallel or antiparallel with respect to each other) in a binary system.  The Backwards One Body ("BOB") waveform $h^{\rm BOB}$ (defined in [this cell](#hBOB)) describes the gravitational wave radiation in the late-inspiral and merger-ringdown stages of this system; we will define $h^{\rm BOB}$ as in [McWilliams,S.T(2019)](https://arxiv.org/pdf/1810.00040.pdf).  There, $h^{\rm BOB}$ is described in terms of the characteristics of final merged black hole. Here, we seek to break up $h^{\rm BOB}$ and document the terms in such a way that the resulting Python code can be used to numerically evaluate it.

Please note that throughout this notebook we adopt the following convention $G = M = c = 1$, where $G$ is the universal gravitational constant, $M = m_1 + m_2$ is the total mass of the binary (**NOT** the mass of the final black hole $M_f$) and, $c$ is the speed of light in vacuum.

Running this notebook to completion will generate a file called v5HM-BOB_unoptimized_merger_ringdown.py. This file contains the Python function v5HM-BOB_unoptimized_merger_ringdown(), which takes as input the initial mass m1, m2 (normalized by the total mass, i.e, $m_! + m_2 = 1$), the values of the normalized spins $\chi_{1,2} = \frac{S_{1,2}}{m_{1,2}^2}$, values for the final mass $M_f$, final spin $\chi_f$, peak waveform frequency $\omega_{22}^{\rm peak}$ .

### Citations
Throughout this module, we will refer to
* [McWilliams,S.T(2019)](https://arxiv.org/pdf/1810.00040.pdf) as BOB
* [Pompili, Buonanno, et al (2023)](https://arxiv.org/pdf/2303.18039.pdf) as PB2023,
* [SEOBNRv5HM-Notes](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf) as V5HM,
* [pySEOBNR documentation/code](https://git.ligo.org/waveforms/software/pyseobnr/) as pySEOBNR.

<a id='outputcreation'></a>

# Step 0: Creating the output directory for SEOBNR \[Back to [top](#toc)\]
$$\label{outputcreation}$$

First we create the output directory for SEOBNR (if it does not already exist):

In [1]:
import sys#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Radiation"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

<a id='phi'></a>

# Step : The waveform phase $\phi$ \[Back to [top](#toc)\]
$$\label{\phi}$$

The waveform phase is given in terms of the orbital phase which is given by equation 10 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    \phi = 2\Phi\\
    \Phi = \textrm{arctan}_{+} + \textrm{arctanh}_{+} - \textrm{arctan}_{-} - \textrm{arctanh}_{-}.
\end{equation*}

Here, $\textrm{arctan}[\textrm{h}]_{\pm}$ is defined in [this cell](#arctanhpm). 

In [2]:
%%writefile $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
phi = 2*Phi
Phi = arctanp + arctanhp - arctanm - arctanhm

Overwriting Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='h'></a>

# Step : The strain amplitude $h$ \[Back to [top](#toc)\]
$$\label{h}$$

The strain amplitude as stated in the discussion below Equation 5 of [BOB](https://arxiv.org/pdf/1810.00040.pdf) :

\begin{equation*}
    h = \frac{A_p}{4\Omega^2}\textrm{sech}\left(\frac{t - t_p}{\tau}\right)
\end{equation*}

Here, $A_p$ is defined in [this cell](#ap), $t_{p}$ is defined in [this cell](#tp), $\Omega$ is defined in [this cell](#omega). The time $t$ and the QNM damping timescale $\tau$ are given as inputs. 

In [3]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
h = (Ap/4/(Omega**2))*(1/sp.cosh((t - tp)/tau))

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='arctanhpm'></a>

# Step : ${\rm arctan}[{\rm h}]_{\pm}$ \[Back to [top](#toc)\]
$$\label{arctanhpm}$$

The phase terms are described in Equation 10 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    {\rm arctan}[{\rm h}]_{\pm} = \kappa_{\pm}\tau\left[ {\rm arctan}[{\rm h}]\left(\frac{\Omega}{\kappa_{\pm}}\right) - {\rm arctan}[{\rm h}]\left(\frac{\Omega_0}{\kappa_{\pm}}\right) \right]
\end{equation*}

Here,

Note: Since $\kappa_{-} = \Omega(t = -\infty)$ will always be lesser than $\Omega$ and $\Omega_0$, $\Omega_0/\kappa_{-}$ and $\Omega/\kappa_{-}$${\rm tanh}^{-1}$ will be outside the domain of the ${\rm arctanh}$ function. However, if we were to explicitly write
$$
    {\rm arctanh}(x) \equiv \frac{1}{2}\ln\left(\frac{1 + x}{1 - x}\right)
$$

Then, one can use the properties of $\ln$ to get

$$
{\rm arctanh}(x) - {\rm arctanh}(y) \equiv \frac{1}{2}\ln\left(\frac{\frac{1 + x}{1 - x}}{\frac{1+y}{1-y}}\right)
$$

which would resolve the domain error issue.

In [4]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
arctanp = kappap*tau*( sp.atan2(Omega,kappap) - sp.atan2(Omega0,kappap) )
arctanm = kappam*tau*( sp.atan2(Omega,kappam) - sp.atan2(Omega0,kappam) )
arctanhp = kappap*tau*( arctanh_Omega_over_kappap_minus_Omega0_over_kappap )
arctanhm = kappam*tau*( arctanh_Omega_over_kappam_minus_Omega0_over_kappam )
arctanh_Omega_over_kappap_minus_Omega0_over_kappap = sp.Rational(1,2)*sp.log( ( (1 + Omega/kappap)/(1 - Omega/kappap) ) / ( (1 + Omega0/kappap)/(1 - Omega0/kappap)) )
arctanh_Omega_over_kappam_minus_Omega0_over_kappam = sp.Rational(1,2)*sp.log( ( (1 + Omega/kappam)/(1 - Omega/kappam) ) / ( (1 + Omega0/kappam)/(1 - Omega0/kappam)) )

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='omega'></a>

# Step : The Effective Orbital Frequency $\Omega$ \[Back to [top](#toc)\]
$$\label{omega}$$

The effective orbital frequency is given defined in Equation 7 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):

\begin{equation*}
    \Omega = \left\{ \Omega_0^4 + k\left[ \tanh\left(\frac{t - t_p}{\tau}\right) - \tanh\left(\frac{t_0 - t_p}{\tau}\right) \right] \right\}^{1/4}.
\end{equation*}

Where, the reference frequency $\Omega_0$ is defined in [this cell](#omega0), $k$ is defined in [this cell], and $t_p$ is defined in [this cell](tp). The times $t$, $t_0$, and the QNM damping timescale $\tau$ are given as inputs. 

In [5]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
Omega = ( Omega0**4 + k*( sp.tanh((t - tp)/tau) - sp.tanh((t0 - tp)/tau) ) )**(1/4)

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='kappapm'></a>

# Step : $\kappa_{\pm}$ \[Back to [top](#toc)\]
$$\label{kappapm}$$

The coefficient $\kappa_{\pm}$ is expressed in Equation 10 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    \kappa_{\pm} = \Omega(t = \pm \infty) = \left\{ \Omega_0^4 \pm k\left[ 1 \mp \tanh\left(\frac{t_0 - t_p}{\tau}\right) \right] \right\}^{1/4}.
\end{equation*}

Here, $\Omega_0$ is defined in [this cell](#omega0), $t_p$ is defined in [this cell](#tp) and $k$ is defined [in this cell](#k).

In [6]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
kappap = (Omega0**4 + k*( 1 - sp.tanh((t0 - tp)/tau) ))**(1/4)
kappam = (Omega0**4 - k*( 1 + sp.tanh((t0 - tp)/tau) ))**(1/4)

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='k'></a>

# Step : $k$ \[Back to [top](#toc)\]
$$\label{bkerrnpa}$$

The coefficient $k$ is expressed in Equation 8 of [BOB](https://arxiv.org/pdf/1810.00040.pdf):
\begin{equation*}
    k = \left( \frac{\Omega_{\rm QNM}^4 - \Omega_0^4}{1 - \tanh\left[(t_0 - t_p)/\tau\right]} \right).
\end{equation*}

Here, $\Omega_0$ is defined in [this cell](#omega0), $t_p$ is defined in [this cell](#t_p). The reference time $t_0$, the QNM frequency $\Omega_{\rm QNM}$, and the QNM damping time $\tau$ are given as inputs.

In [7]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
k = (OmegaQNM**4 - Omega0**4)/(1 - sp.tanh((t0 - tp)/tau))

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='ap'></a>

# Step : $A_p$ \[Back to [top](#toc)\]
$$\label{ap}$$

The peak $\psi_4$ amplitude is calculated by matching the strain amplitude $h$ at the peak strain time to it's value given by numerical relativity fits:
\begin{equation*}
    A_p = h_{\rm NR}\omega^2_{\rm NR}\cosh\left( \frac{t_0 - t_p}{\tau} \right).
\end{equation*}

Here, $h_{\rm NR}$ and $\omega_{rm NR}$ are the peak strain ampitude and the corresponding frequency as determined by numerical relativity fits. $t_0$ and $\tau$ are given as inputs and $t_p$ is defined in [this cell](#tp).

In [8]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
Ap = hNR*(omegaNR**2)*sp.cosh((t0 - tp)/tau)

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='tp'></a>

# Step : $t_p$ \[Back to [top](#toc)\]
$$\label{tp}$$

The peak $\psi_4$ time $t_p$ is obtained by solving for it under the condition that the reference time corresponds to the peak of the strain amplitude $\dot{h}(t_0) = 0$ to give:
\begin{equation*}
    t_p = t_0 - 2\tau\ln\left(\frac{\Omega_0}{\Omega_{\rm QNM}}\right).
\end{equation*}

Here, $\Omega_0$ is defined in [this cell](#omega0). The QNM frequency $\Omega_{\rm QNM}$, the QNM damping time $\tau$, and the reference time $t_0$ are given as inputs.

In [9]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
tp = t0 - 2*tau*sp.log(Omega0/OmegaQNM)

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='omegaqnm'></a>

# Step : $\Omega_{\rm QNM}$ \[Back to [top](#toc)\]
$$\label{omegaqnm}$$

The "orbital" QNM frequency $\Omega_{QNM}$ is expressed in terms of the QNM frequency:
\begin{equation*}
    \Omega_{\rm QNM} = \frac{\omega_{\rm QNM}}{2}.
\end{equation*}

Here, $\omega_{\rm QNM}$ is given as an input.

In [10]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
OmegaQNM = omegaQNM/2

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


<a id='omega0'></a>

# Step : $\Omega_0$ \[Back to [top](#toc)\]
$$\label{omega0}$$

The reference effective orbital frequency $\Omega_{0}$ is expressed in terms of the waveform frequency corresponding to the peak of the strain amplitude:
\begin{equation*}
    \Omega_0 = \frac{\omega_{\rm NR}}{2}.
\end{equation*}

Here, $\omega_{\rm NR}$ is given as an input.

In [11]:
%%writefile -a $Ccodesdir/v5HM-BOB_merger_ringdown_on_top.txt
Omega0 = omegaNR/2

Appending to Radiation/v5HM-BOB_merger_ringdown_on_top.txt


# Step : Saving the expressions

Up till now, the expressions required for the Hamiltonian have been stored in a .txt file. for the sake of readability, some of the expressions have been written in more than one line. 

Therefore, we save the expressions in one-line format to parse into sympy for generating optimized code or derivative expressions.

In [12]:
import os 
with open(os.path.join(Ccodesdir,"v5HM-BOB_merger_ringdown_on_top-oneline.txt"), "w") as output:
    count = 0
    # Read output of this notebook
    for line in list(open(os.path.join(Ccodesdir,"v5HM-BOB_merger_ringdown_on_top.txt"),"r")):
        # Read the first line
        if count == 0:
            prevline=line
        #Check if prevline is a complete expression
        elif "=" in prevline and "=" in line:
            output.write("%s\n" % prevline.strip('\n'))
            prevline=line
        # Check if line needs to be adjoined to prevline
        elif "=" in prevline and not "=" in line:
            prevline = prevline.strip('\n')
            prevline = (prevline+line).replace(" ","")
        # Be sure to print the last line.
        if count == len(list(open(os.path.join(Ccodesdir,"v5HM-BOB_merger_ringdown_on_top.txt"))))-1:
            if not "=" in line:
                print("ERROR. Algorithm not robust if there is no equals sign on the final line. Sorry.")
                sys.exit(1)
            else:
                output.write("%s" % line)
        count = count + 1

with open(os.path.join(Ccodesdir,"v5HM-BOB_merger_ringdown_on_bottom.txt"), "w") as output:
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM-BOB_merger_ringdown_on_top-oneline.txt"),"r"))):
        output.write("%s\n"%line.rstrip())


# Generate unoptimized python code

In this step, we will simply store the Hamiltonian expressions in a python module as is in order to run preliminary validation tests. In addition, we will compare the results of our Hamiltonian function with that of the pyseobnr Hamiltonian function.

In [13]:
with open(os.path.join(Ccodesdir,"v5HM_BOB_unoptimized_merger_ringdown.py"), "w") as output:
    output.write("import numpy as np\ndef v5HM_BOB_unoptimized_merger_ringdown(t,t0,hNR,omegaNR,omegaQNM,tau):\n")
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM-BOB_merger_ringdown_on_top-oneline.txt"),"r"))):
        output.write("    %s\n" % line.rstrip().replace("sp.tanh", "np.tanh").replace("sp.Rational",
                                "np.divide").replace("sp.log","np.log").replace("sp.atan","np.arctan").replace("sp.cosh","np.cosh").replace("sp.cos","np.cos").replace("sp.sin","np.sin"))
    output.write("    return h,np.unwrap(phi)")

# Generate derivatives for NQC's

In this step, we will write the BOB expressions into a sympy function that will use `sympy.diff` and `sympy.pycode` to return the unoptimized python code for the following variables needed for the NQCs:

1. The amplitude $h$ and its time derivatives $\dot{h}$ and $\ddot{h}$
1. The frequency $\omega \equiv 2\Omega$ and its time derivative $\dot{\omega}$

We will then call this function and store the pycode into a numpy function for the purpose of calculating NQC coefficients.

In [14]:
Outputdir = "IMR"
# Then create an output directory in case it does not exist
cmd.mkdir(Outputdir)

with open(os.path.join(Outputdir,"v5HM_BOB_generate_unoptimized_nqc_terms.py"), "w") as output:
    output.write("import sympy as sp\ndef v5HM_BOB_generate_unoptimized_nqc_terms():\n")
    output.write("    t,t0,hNR,omegaNR,omegaQNM,tau = sp.symbols('t t0 hNR omegaNR omegaQNM tau',real = True)\n")
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM-BOB_merger_ringdown_on_top-oneline.txt"),"r"))):
        output.write("    %s\n" % line.rstrip())
    output.write("    hdot = sp.diff(h,t)\n")
    output.write("    hddot = sp.diff(hdot,t)\n")
    output.write("    omega = 2*Omega\n")
    output.write("    omegadot = sp.diff(omega,t)\n")
    output.write("    return sp.pycode(h),sp.pycode(hdot),sp.pycode(hddot),sp.pycode(omega),sp.pycode(omegadot)")
    
from IMR.v5HM_BOB_generate_unoptimized_nqc_terms import v5HM_BOB_generate_unoptimized_nqc_terms

h, hdot, hddot, omega, omegadot = v5HM_BOB_generate_unoptimized_nqc_terms()

terms_lhs = ["h", "hdot", "hddot", "omega", "omegadot"]
terms_rhs = [h, hdot, hddot, omega, omegadot]
with open(os.path.join(Outputdir,"v5HM_BOB_unoptimized_nqc_terms.py"), "w") as output:
    output.write("import numpy as np\ndef v5HM_BOB_unoptimized_nqc_terms(t,t0,hNR,omegaNR,omegaQNM,tau):\n")
    for i in range(len(terms_lhs)):
        output.write(f"    {terms_lhs[i]} = %s \n" % terms_rhs[i].replace('math.','np.'))
    output.write(f"    return [h, hdot, hddot], [omega, omegadot]")